In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from conf.tmhelix import kink_analysis
from plot.plot_utilities import hist1d, hist2d, edgeformat

In [ ]:
%store -r traj_ids
%store -r traj_ids_closed
all_traj_ids = np.concatenate([traj_ids, traj_ids_closed])

%store -r states_df

%store -r color_assign
%store -r map_assign

# All angles of interest

In [ ]:
states = sorted(states_df['tmpc1v2'].unique())

In [ ]:
def kink2d_plot(axs, phi, theta, range2d, dens2dmax=0.0066, colors='black'):
    hist = hist2d(phi, theta, bins=60, range=range2d)
    hist.hist2d_contour(axs, levels=np.linspace(0,dens2dmax,15)[1:], colors=colors, linewidths=1)
    hist.hist2d_contourf(axs, levels=[0,1], colors=colors, alpha=0.1)
    return hist

def theta_plot(axs, theta, range, color='black'):
    hist = hist1d(theta, bins=60, range=range)
    axs.plot(-hist.dens, hist.plot_edges, color=color)
    return hist

def phi_plot(axs, phi, range, color='black'):
    hist = hist1d(phi, bins=60, range=range)
    hist.plot(axs, color=color)
    return hist

# Comprehensive angle phot
def angle_plot_cphs(phi, theta, phi_range, theta_range, phi_densmax, theta_densmax, dens2dmax=0.0066):
    fig, Axs = plt.subplots(2, 2, gridspec_kw={'wspace': 0, 'hspace': 0, 'width_ratios': [1, 3], 'height_ratios': [1, 3]})
    edgeformat(Axs)

    for s in states:
        states_select = (states_df['tmpc1v2'] == s)

        axs = Axs[1,1]
        hist = kink2d_plot(axs, 
                           phi[states_select], theta[states_select], 
                           range2d=[phi_range, theta_range], 
                           dens2dmax=dens2dmax, 
                           colors=color_assign[s])
        # print(hist.densmax)
        axs.set_xlim(*phi_range)
        axs.set_ylim(*theta_range)
        axs.grid(True, ls='--')
        axs.set_xlabel(r"$\phi$ [deg]", fontsize=16)
        # Hide y-ticks for this region
        axs.set_yticklabels([])
        axs.set_xticks(np.linspace(phi_range[0], phi_range[1],7))

        axs = Axs[1,0]
        hist = theta_plot(axs, 
                          theta[states_select], 
                          range=theta_range, 
                          color=color_assign[s])
        axs.set_xlim(-theta_densmax,0)
        axs.set_ylim(*theta_range)
        axs.grid(True, ls='--')
        axs.set_ylabel(r"$\theta$ [deg]", fontsize=16)
        # Hide x-ticks for this region
        axs.set_xticklabels([])

        axs = Axs[0,1]
        hist = phi_plot(axs, 
                        phi[states_select], 
                        range=phi_range, 
                        color=color_assign[s])
        axs.set_xlim(*phi_range)
        axs.set_ylim(0,phi_densmax)
        axs.grid(True, ls='--')  
        # Hide x and y-ticks for this region  
        axs.set_xticklabels([])
        axs.set_yticklabels([])
        axs.set_xticks(np.linspace(phi_range[0], phi_range[1],7))

        # Make this region completely empty
        axs = Axs[0,0]
        axs.axis('off')
    
    return Axs

### P99

In [ ]:
z_seg = [85,96]
w_seg = [97,108]
phi_range = [-180,180]
theta_range = [0,60]

main_tm1top = kink_analysis(traj_ids, z_seg=z_seg, w_seg=w_seg)
theta, phi = main_tm1top.calculate()

Axs = angle_plot_cphs(phi, theta, phi_range=phi_range, theta_range=theta_range, phi_densmax=0.05, theta_densmax=0.15, dens2dmax=0.0066)

open_select = (states_df['tmpc1v2'] == 3).values
open_theta = theta[open_select]
open_phi = phi[open_select]
print(f"MD-open Theta: {open_theta.mean():.2f}+-{open_theta.std():.2f}")
print(f"MD-open Phi: {open_phi.mean():.2f}+-{open_phi.std():.2f}")

closed_select = ((states_df['tmpc1v2'] != 3) & (states_df['tmpc1v2'] != 1)).values
closed_theta = theta[closed_select]
closed_phi = phi[closed_select]
print(f"MD-closed Theta: {closed_theta.mean():.2f}+-{closed_theta.std():.2f}")
print(f"MD-closed Phi: {closed_phi.mean():.2f}+-{closed_phi.std():.2f}")

### G85

In [ ]:
z_seg = [78,85]
w_seg = [85,96]
phi_range = [-360,0]
theta_range = [0,60]

main_tm1mid = kink_analysis(traj_ids, z_seg=z_seg, w_seg=w_seg)

# phi correction; it is periodic
# makes plot look better
def correct_phi(phi):
    if phi > 0:
        return phi - 360
    return phi
theta, phi = main_tm1mid.calculate(periodic_correction=correct_phi)

Axs = angle_plot_cphs(phi, theta, phi_range=phi_range, theta_range=theta_range, phi_densmax=0.05, theta_densmax=0.15, dens2dmax=0.0066)

open_select = (states_df['tmpc1v2'] == 3).values
open_theta = theta[open_select]
open_phi = phi[open_select]
print(f"MD-open Theta: {open_theta.mean():.2f}+-{open_theta.std():.2f}")
print(f"MD-open Phi: {open_phi.mean():.2f}+-{open_phi.std():.2f}")

closed_select = ((states_df['tmpc1v2'] != 3) & (states_df['tmpc1v2'] != 1)).values
closed_theta = theta[closed_select]
closed_phi = phi[closed_select]
print(f"MD-closed Theta: {closed_theta.mean():.2f}+-{closed_theta.std():.2f}")
print(f"MD-closed Phi: {closed_phi.mean():.2f}+-{closed_phi.std():.2f}")

### G126

In [ ]:
z_seg = [129,138]
w_seg = [117,128]
phi_range = [-180,180]
theta_range = [0,60]

# phi correction; it is periodic
# makes plot look better
def correct_phi(phi):
    if phi < -180:
        return phi + 360
    return phi

main_tm2top = kink_analysis(traj_ids, z_seg=z_seg, w_seg=w_seg)
theta, phi = main_tm2top.calculate(periodic_correction=correct_phi)

Axs = angle_plot_cphs(phi, theta, phi_range=phi_range, theta_range=theta_range, phi_densmax=0.05, theta_densmax=0.15, dens2dmax=0.0066)

open_select = (states_df['tmpc1v2'] == 3).values
open_theta = theta[open_select]
open_phi = phi[open_select]
print(f"MD-open Theta: {open_theta.mean():.2f}+-{open_theta.std():.2f}")
print(f"MD-open Phi: {open_phi.mean():.2f}+-{open_phi.std():.2f}")

closed_select = ((states_df['tmpc1v2'] != 3) & (states_df['tmpc1v2'] != 1)).values
closed_theta = theta[closed_select]
closed_phi = phi[closed_select]
print(f"MD-closed Theta: {closed_theta.mean():.2f}+-{closed_theta.std():.2f}")
print(f"MD-closed Phi: {closed_phi.mean():.2f}+-{closed_phi.std():.2f}")

### P140

In [ ]:
z_seg = [140,149]
w_seg = [129,138]
phi_range = [-360,0]
theta_range = [0,60]

main_tm2mid = kink_analysis(traj_ids, z_seg=z_seg, w_seg=w_seg)

# phi correction; it is periodic
# makes plot look better
def correct_phi(phi):
    if phi > 0:
        return phi - 360
    return phi
theta, phi = main_tm2mid.calculate(periodic_correction=correct_phi)

Axs = angle_plot_cphs(phi, theta, phi_range=phi_range, theta_range=theta_range, phi_densmax=0.05, theta_densmax=0.15, dens2dmax=0.0066)

open_select = (states_df['tmpc1v2'] == 3).values
open_theta = theta[open_select]
open_phi = phi[open_select]
print(f"MD-open Theta: {open_theta.mean():.2f}+-{open_theta.std():.2f}")
print(f"MD-open Phi: {open_phi.mean():.2f}+-{open_phi.std():.2f}")

closed_select = ((states_df['tmpc1v2'] != 3) & (states_df['tmpc1v2'] != 1)).values
closed_theta = theta[closed_select]
closed_phi = phi[closed_select]
print(f"MD-closed Theta: {closed_theta.mean():.2f}+-{closed_theta.std():.2f}")
print(f"MD-closed Phi: {closed_phi.mean():.2f}+-{closed_phi.std():.2f}")

### W1098

In [ ]:
z_seg = [1077,1097]
w_seg = [1098,1118]
phi_range = [60, 420]
theta_range = [0,60]

main_tm11mid = kink_analysis(traj_ids, z_seg=z_seg, w_seg=w_seg)

# phi correction; it is periodic
# makes plot look better
def correct_phi(phi):
    if phi < 60:
        return phi + 360
    return phi
theta, phi = main_tm11mid.calculate(periodic_correction=correct_phi)

Axs = angle_plot_cphs(phi, theta, phi_range=phi_range, theta_range=theta_range, phi_densmax=0.05, theta_densmax=0.15, dens2dmax=0.0066)

open_select = (states_df['tmpc1v2'] == 3).values
open_theta = theta[open_select]
open_phi = phi[open_select]
print(f"MD-open Theta: {open_theta.mean():.2f}+-{open_theta.std():.2f}")
print(f"MD-open Phi: {open_phi.mean():.2f}+-{open_phi.std():.2f}")

closed_select = ((states_df['tmpc1v2'] != 3) & (states_df['tmpc1v2'] != 1)).values
closed_theta = np.nan_to_num(theta[closed_select])
closed_phi = phi[closed_select]
print(f"MD-closed Theta: {closed_theta.mean():.2f}+-{closed_theta.std():.2f}")
print(f"MD-closed Phi: {closed_phi.mean():.2f}+-{closed_phi.std():.2f}")

# Angle vs PC1

In [ ]:
%store -r pca_df

### P99

In [ ]:
xrange = [0,60]
yrange = [-60,60]
nbins = 50


fig, axs = plt.subplots()
edgeformat(axs)

angle_collect = []
pc1_collect = []

# for t in traj_grps["2"]:
# for t in traj_set:
for t in traj_ids:
    sel = states_df.query('traj_id == @t').index
    
    axs.scatter(main_tm1top.theta[sel], pca_df.iloc[sel]['pc1'].values, c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
    
    angle_collect.append(main_tm1top.theta[sel])
    pc1_collect.append(pca_df.iloc[sel]['pc1'].values)
    
axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')

angle_collect = np.concatenate(angle_collect)
pc1_collect = np.concatenate(pc1_collect)



fig, axs = plt.subplots()
edgeformat(axs)

hist2d(angle_collect, pc1_collect, bins=nbins, range=[xrange,yrange]).dens2d_preset2(axs, no_last_contour_line=False)

axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')
axs.set_xlabel(r"$\theta$ [deg]", fontsize=16)
axs.set_ylabel(r"PC1 [$\mathrm{\AA}$]", fontsize=16)

# savefig("TM1topkink_vs_pc1.pdf")

### G85

In [ ]:
xrange = [-240,-120]
yrange = [-60,60]
nbins = 50


fig, axs = plt.subplots()
edgeformat(axs)

angle_collect = []
pc1_collect = []

# for t in traj_grps["2"]:
# for t in traj_set:
for t in traj_ids:
    sel = states_df.query('traj_id == @t').index
    
    angles = main_tm1mid.phi[sel]
    pc1vals = pca_df.iloc[sel]['pc1'].values
    
    axs.scatter(angles, pc1vals, c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
    
    angle_collect.append(angles)
    pc1_collect.append(pc1vals)
    
axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')

angle_collect = np.concatenate(angle_collect)
pc1_collect = np.concatenate(pc1_collect)



fig, axs = plt.subplots()
edgeformat(axs)

hist2d(angle_collect, pc1_collect, bins=nbins, range=[xrange,yrange]).dens2d_preset2(axs, no_last_contour_line=False)

axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')
axs.set_xlabel(r"$\phi$ [deg]", fontsize=16)
axs.set_ylabel(r"PC1 [$\mathrm{\AA}$]", fontsize=16)

# savefig("TM1midswivel_vs_pc1.pdf")

### G126

In [ ]:
xrange = [-180,0]
yrange = [-60,60]
nbins = 50


fig, axs = plt.subplots()
edgeformat(axs)

angle_collect = []
pc1_collect = []

# for t in traj_grps["2"]:
# for t in traj_set:
for t in traj_ids:
    sel = states_df.query('traj_id == @t').index
    
    angles = main_tm2top.phi[sel]
    pc1vals = pca_df.iloc[sel]['pc1'].values
    
    axs.scatter(angles, pc1vals, c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
    
    angle_collect.append(angles)
    pc1_collect.append(pc1vals)
    
axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')

angle_collect = np.concatenate(angle_collect)
pc1_collect = np.concatenate(pc1_collect)



fig, axs = plt.subplots()
edgeformat(axs)

hist2d(angle_collect, pc1_collect, bins=nbins, range=[xrange,yrange]).dens2d_preset2(axs, no_last_contour_line=False)

axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')
axs.set_xlabel(r"$\phi$ [deg]", fontsize=16)
axs.set_ylabel(r"PC1 [$\mathrm{\AA}$]", fontsize=16)

# savefig("TM2topswivel_vs_pc1.pdf")

### P140

In [ ]:
xrange = [-200,-80]
yrange = [-60,60]
nbins = 50


fig, axs = plt.subplots()
edgeformat(axs)

angle_collect = []
pc1_collect = []

# for t in traj_grps["2"]:
# for t in traj_set:
for t in traj_ids:
    sel = states_df.query('traj_id == @t').index
    
    angles = main_tm2mid.phi[sel]
    pc1vals = pca_df.iloc[sel]['pc1'].values
    
    axs.scatter(angles, pc1vals, c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
    
    angle_collect.append(angles)
    pc1_collect.append(pc1vals)
    
axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')

angle_collect = np.concatenate(angle_collect)
pc1_collect = np.concatenate(pc1_collect)



fig, axs = plt.subplots()
edgeformat(axs)

hist2d(angle_collect, pc1_collect, bins=nbins, range=[xrange,yrange]).dens2d_preset2(axs, no_last_contour_line=False)

axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')
axs.set_xlabel(r"$\phi$ [deg]", fontsize=16)
axs.set_ylabel(r"PC1 [$\mathrm{\AA}$]", fontsize=16)

# savefig("TM2midswivel_vs_pc1.pdf")

### W1098

In [ ]:
xrange = [0,25]
yrange = [-60,60]
nbins = 50


fig, axs = plt.subplots()
edgeformat(axs)

angle_collect = []
pc1_collect = []

# for t in traj_grps["2"]:
# for t in traj_set:
for t in traj_ids:
    sel = states_df.query('traj_id == @t').index
    
    angles = main_tm11mid.theta[sel]
    pc1vals = pca_df.iloc[sel]['pc1'].values
    
    axs.scatter(angles, pc1vals, c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
    
    angle_collect.append(angles)
    pc1_collect.append(pc1vals)
    
axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')

angle_collect = np.concatenate(angle_collect)
pc1_collect = np.concatenate(pc1_collect)



fig, axs = plt.subplots()
edgeformat(axs)

hist2d(angle_collect, pc1_collect, bins=nbins, range=[xrange,yrange]).dens2d_preset2(axs, no_last_contour_line=False)

axs.grid(True)
axs.set_xlim(*xrange)
axs.set_ylim(*yrange)
axs.set_aspect(np.diff(xrange)/np.diff(yrange)[0], adjustable='box', anchor='C')
axs.set_xlabel(r"$\theta$ [deg]", fontsize=16)
axs.set_ylabel(r"PC1 [$\mathrm{\AA}$]", fontsize=16)

# savefig("TM11midkink_vs_pc1.pdf")

In [ ]:
t = 69

fig, axs = plt.subplots(1,4, figsize=(16,12), sharey=True)

sel = states_df.query('traj_id == @t').index

axs[0].scatter(main_tm2top.phi[sel], main_tm1mid.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[0].grid(True)
axs[0].set_ylim(-240,-120)
axs[0].set_xlim(-180,0)
axs[0].set_aspect(180/120, adjustable='box', anchor='C')

axs[1].scatter(main_tm2mid.phi[sel], main_tm1mid.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[1].grid(True)
axs[1].set_ylim(-240,-120)
axs[1].set_xlim(-200,-80)
axs[1].set_aspect(120/120, adjustable='box', anchor='C')

axs[2].scatter(main_tm1top.theta[sel], main_tm1mid.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[2].grid(True)
axs[2].set_ylim(-240,-120)
axs[2].set_xlim(0,60)
axs[2].set_aspect(60/120, adjustable='box', anchor='C')

axs[3].scatter(main_tm11mid.theta[sel], main_tm1mid.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[3].grid(True)
axs[3].set_ylim(-240,-120)
axs[3].set_xlim(0,25)
axs[3].set_aspect(25/120, adjustable='box', anchor='C')


fig, axs = plt.subplots(1,3, figsize=(12,12), sharey=True)

axs[0].scatter(main_tm2mid.phi[sel], main_tm2top.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[0].grid(True)
axs[0].set_ylim(-180,0)
axs[0].set_xlim(-200,-60)
axs[0].set_aspect(140/180, adjustable='box', anchor='C')

axs[1].scatter(main_tm1top.theta[sel], main_tm2top.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[1].grid(True)
axs[1].set_ylim(-180,0)
axs[1].set_xlim(0,60)
axs[1].set_aspect(60/180, adjustable='box', anchor='C')

axs[2].scatter(main_tm11mid.theta[sel], main_tm2top.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[2].grid(True)
axs[2].set_ylim(-180,0)
axs[2].set_xlim(0,25)
axs[2].set_aspect(25/180, adjustable='box', anchor='C')


fig, axs = plt.subplots(1,2, figsize=(8,12), sharey=True)

axs[0].scatter(main_tm1top.theta[sel], main_tm2mid.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[0].grid(True)
axs[0].set_xlim(0,60)
axs[0].set_ylim(-200,-60)
axs[0].set_aspect(60/140, adjustable='box', anchor='C')

axs[1].scatter(main_tm11mid.theta[sel], main_tm2mid.phi[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs[1].grid(True)
axs[1].set_xlim(0,25)
axs[1].set_ylim(-200,-60)
axs[1].set_aspect(25/140, adjustable='box', anchor='C')


fig, axs = plt.subplots(1,1, figsize=(4,12))

axs.scatter(main_tm11mid.theta[sel], main_tm1top.theta[sel], c=np.arange(len(main_tm2mid.theta[sel])), cmap='viridis', s=6)
axs.grid(True)
axs.set_ylim(0,60)
axs.set_xlim(0,25)
axs.set_aspect(25/60, adjustable='box', anchor='C')